In [1]:
import joblib
import librosa
import numpy as np
import gradio as gr

gmm_male_enh = joblib.load('gmm_male_enh.pkl')
gmm_female_enh = joblib.load('gmm_female_enh.pkl')

print("Modele enhanced încărcate cu succes!")

def extract_enhanced_features(audio):
    mfcc = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std  = np.std(mfcc, axis=1)
    
    f0, voiced_flag, _ = librosa.pyin(
        audio,
        fmin=librosa.note_to_hz('C2'),
        fmax=librosa.note_to_hz('C7'),
        sr=16000,
        frame_length=1024,
        hop_length=256,
        fill_na=np.nan
    )
    voiced_f0 = f0[voiced_flag & ~np.isnan(f0)]
    pitch_mean = np.mean(voiced_f0) if len(voiced_f0) > 0 else 120.0
    pitch_std  = np.std(voiced_f0) if len(voiced_f0) > 0 else 20.0
    
    return np.concatenate([mfcc_mean, mfcc_std, [pitch_mean, pitch_std]])

def predict_gender(audio, sr=16000):
    if sr != 16000:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
    
    feat = extract_enhanced_features(audio)
    score_m = gmm_male_enh.score(feat.reshape(1, -1))
    score_f = gmm_female_enh.score(feat.reshape(1, -1))
    
    pred = 'female' if score_f > score_m else 'male'
    confidence_proxy = abs(score_f - score_m)
    
    return {
        'gender': pred,
        'confidence_proxy': confidence_proxy,
        'features_dim': len(feat)
    }

# Demo Gradio
def gradio_predict(audio):
    if audio is None:
        return "No audio received."
    
    sr, wav = audio
    
    if wav.dtype == np.int16:
        wav = wav.astype(np.float32) / 32768.0
    elif wav.dtype != np.float32:
        wav = wav.astype(np.float32)
    
    if np.max(np.abs(wav)) > 0:
        wav = wav / np.max(np.abs(wav))
    
    result = predict_gender(wav, sr)
    return f"Gender: **{result['gender'].upper()}**\nConfidence proxy: {result['confidence_proxy']:.4f}\nFeatures: {result['features_dim']}-dim"

demo = gr.Interface(
    fn=gradio_predict,
    inputs=gr.Audio(sources=["microphone"], type="numpy"),
    outputs="text",
    title="Gender Recognition Live Demo - Giulia Stefania",
    description="Press 'Record', speak clearly for 3–5 seconds, then 'Stop'."
)

demo.launch(share=False)

/Users/giulia/Desktop/master/bd/proiect/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Modele enhanced încărcate cu succes!
* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
